<a href="https://colab.research.google.com/github/jameswilsenach/LMforOntology/blob/main/MaskedLanguage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.4 MB/s eta 0:00:00


In [ ]:

from transformers import BertTokenizer, BertForMaskedLM
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text = ("Humans are animals. A cat is a blank.")
inputs = tokenizer(text, return_tensors='pt')

In [ ]:
# create random array of floats in equal dimension to input_ids
# rand = torch.rand(inputs.input_ids.shape)
mask_arr = torch.zeros(inputs.input_ids.shape, dtype=torch.bool)
# where the random array is less than 0.15, we set true 
mask_arr[0,-3] = True


In [ ]:
# create selection from mask_arr
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
selection_id = inputs.input_ids[0,-3]
print(selection_id)
mask_word = tokenizer.decode([selection_id])
print(mask_word)

tensor(8744)
blank


In [ ]:
# apply selection index to inputs.input_ids, adding MASK tokens
inputs.input_ids[0, selection] = 103
tokenizer.decode(inputs.input_ids[0,:])

'[CLS] humans are animals. a cat is a [MASK]. [SEP]'

In [ ]:
# pass inputs as kwarg to model
outputs = model(**inputs)
outputs.keys()

odict_keys(['logits'])

In [ ]:
outputs.keys()


odict_keys(['logits'])

In [ ]:
outputs.logits.shape

torch.Size([1, 12, 30522])

In [ ]:
mask_token_logits = outputs.logits[0, selection, :]

In [ ]:
top_tokens = torch.topk(mask_token_logits, 20, dim=1).indices[0].tolist()

In [ ]:
tokenizer.decode(top_tokens)

'cat human dog person bird man pet mammal creature mouse animal predator species horse being snake thing monkey rat beast'

In [ ]:
for token in top_tokens:
    print(text.replace(mask_word , tokenizer.decode([token])))

Humans are animals. A cat is a cat.
Humans are animals. A cat is a human.
Humans are animals. A cat is a dog.
Humans are animals. A cat is a person.
Humans are animals. A cat is a bird.
Humans are animals. A cat is a man.
Humans are animals. A cat is a pet.
Humans are animals. A cat is a mammal.
Humans are animals. A cat is a creature.
Humans are animals. A cat is a mouse.
Humans are animals. A cat is a animal.
Humans are animals. A cat is a predator.
Humans are animals. A cat is a species.
Humans are animals. A cat is a horse.
Humans are animals. A cat is a being.
Humans are animals. A cat is a snake.
Humans are animals. A cat is a thing.
Humans are animals. A cat is a monkey.
Humans are animals. A cat is a rat.
Humans are animals. A cat is a beast.


In [ ]:
tokens = tokenizer(text, return_tensors='pt')
tokenizer.decode([])

''

In [ ]:
tokenizer.mask_token

'[MASK]'

In [ ]:
tokenizer.decode(inputs.input_ids[0,:])

'[CLS] a cat is a [MASK]. [SEP]'